### Ячейка 1: Импорты и глобальные настройки

In [ ]:
# --- Системные и основные библиотеки ---
import os
import warnings
import pandas as pd
import numpy as np

# --- Визуализация ---
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

# --- Машинное обучение (Scikit-learn) ---
from sklearn.model_selection import train_test_split

# --- Машинное обучение (PyTorch) ---
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

# --- Инструменты для экспериментов ---
import optuna
import mlflow
import mlflow.pytorch

# --- Игнорируем предупреждения для чистоты вывода ---
warnings.filterwarnings('ignore')

# --- Константы ---
DATA_PATH = "data"
FILE_TRAIN = os.path.join(DATA_PATH, "train.csv")
FILE_TEST = os.path.join(DATA_PATH, "test.csv")
SUBMISSION_FILE = 'submission_house_prices.csv'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Настройки для визуализации ---
plt.style.use('ggplot')
rcParams['figure.figsize'] = 12, 8

print("Все библиотеки импортированы.")
print(f"Обучение будет производиться на устройстве: {DEVICE}")
print(f"Optuna version: {optuna.__version__}")
print(f"MLflow version: {mlflow.__version__}")

Все библиотеки импортированы.
Обучение будет производиться на устройстве: cuda
Optuna version: 4.3.0
MLflow version: 3.1.0


In [ ]:
# Ячейка 0: Диагностика окружения
import sys
import optuna
print(f"Путь к Python, который использует ядро: {sys.executable}")
print(f"Версия Optuna, видимая ядру: {optuna.__version__}")

Путь к Python, который использует ядро: /usr/bin/python
Версия Optuna, видимая ядру: 4.3.0


### Ячейка 2: Конфигурация эксперимента

In [ ]:
# --- Общие настройки эксперимента ---
EXPERIMENT_NAME = "House Prices - Optuna & MLflow"
N_TRIALS = 200 # Количество экспериментов для Optuna
EPOCHS = 240  # Фиксированное количество эпох для каждого эксперимента

# --- Базовые параметры (могут быть переопределены Optuna) ---
class Config:
    # Параметры DataLoader
    BATCH_SIZE = 128
    
    # Параметры оптимизатора
    LEARNING_RATE = 0.005
    WEIGHT_DECAY = 0.01
    
    # Параметры планировщика
    SCHEDULER_PATIENCE = 30
    SCHEDULER_FACTOR = 0.5
    
    # Параметры архитектуры модели
    # Список с количеством нейронов в каждом скрытом слое
    HIDDEN_LAYERS = [128, 64] 
    # Список со значением Dropout для каждого скрытого слоя
    DROPOUT_RATES = [0.4, 0.4]

print("Конфигурация эксперимента задана.")
print(f"Optuna проведет {N_TRIALS} экспериментов.")

Конфигурация эксперимента задана.
Optuna проведет 200 экспериментов.


### Ячейка 3: Комплексная обработка данных

In [ ]:
# --- ЗАГРУЗКА ДАННЫХ ---
try:
    df_train = pd.read_csv(FILE_TRAIN)
    df_test = pd.read_csv(FILE_TEST)
    df_test_original = df_test.copy()
    print("Данные успешно загружены.")
except FileNotFoundError:
    print(f"Ошибка: Файлы не найдены в директории '{DATA_PATH}'.")

# --- ПОЛНЫЙ ПАЙПЛАЙН ПРЕДОБРАБОТКИ ---

# 1. Подготовка
df_train_proc = df_train.copy()
df_test_proc = df_test.copy()
train_target = df_train_proc['SalePrice'].copy()
df_train_proc.drop('SalePrice', axis=1, inplace=True)
train_rows = len(df_train_proc)
df_combined = pd.concat([df_train_proc, df_test_proc], ignore_index=True)


# --- ЭТАП 2: БАЗОВАЯ ОЧИСТКА И ЗАПОЛНЕНИЕ ПРОПУСКОВ ---

# 2.1 Рассчитываем медианы на ОБЪЕДИНЕННЫХ данных для большей точности
lotfrontage_median = df_combined['LotFrontage'].median()
garageyrblt_median = df_combined['GarageYrBlt'].median()

# 2.2 Заполняем пропуски
df_combined['LotFrontage'].fillna(lotfrontage_median, inplace=True)
df_combined['GarageYrBlt'].fillna(garageyrblt_median, inplace=True)

# 2.3 Заполняем нулями столбцы, где NA означает "отсутствие" объекта
cols_fill_zero = [
    'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
    'GarageCars', 'GarageArea', 'BsmtFullBath', 'BsmtHalfBath'
]
for col in cols_fill_zero:
    df_combined[col].fillna(0, inplace=True)

# 2.4 Заполняем пропуски в категориальных признаках строкой 'NA'
cat_cols_fill_na = [
    'Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
    'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'PoolQC', 'Fence', 'MiscFeature', 'MasVnrType', 'Electrical', 'MSZoning',
    'Utilities', 'Exterior1st', 'Exterior2nd', 'KitchenQual', 'Functional', 'SaleType'
]
for col in cat_cols_fill_na:
    df_combined[col].fillna('NA', inplace=True)

# 2.5 Исправляем опечатки, замеченные при анализе данных
df_combined['Exterior2nd'].replace({"CmentBd": "CemntBd", "Wd Shng": "WdShing"}, inplace=True)


# --- ЭТАП 3: ИНЖИНИРИНГ ПРИЗНАКОВ (FEATURE ENGINEERING) ---

# 3.1 Временные признаки (давность событий относительно 2010 года)
df_combined['YearBuildAgo'] = 2010 - df_combined['YearBuilt']
df_combined['YearRemodAddAgo'] = 2010 - df_combined['YearRemodAdd']
df_combined['GarageYrBltAgo'] = 2010 - df_combined['GarageYrBlt']
df_combined['MoSoldAgo'] = 12 - df_combined['MoSold'] + 12 * (2010 - df_combined['YrSold'])

# 3.2 Порядковое кодирование (Ordinal Encoding)
qual_map = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
bsmt_fin_map = {'GLQ': 5, 'ALQ': 4, 'BLQ': 3, 'Rec': 3.5, 'LwQ': 2, 'Unf': 1, 'NA': 0}
bsmt_exp_map = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0}
garage_fin_map = {'Fin': 3, 'RFn': 2, 'Unf': 1, 'NA': 0}
functional_map = {'Typ': 0, 'Min1': 2, 'Min2': 1, 'Mod': 3, 'Maj1': 4, 'Maj2': 5, 'Sev': 6, 'NA': 0}
paved_drive_map = {'Y': 0, 'P': 1, 'N': 2}
electrical_map = {'SBrkr': 0, 'FuseA': 1, 'FuseF': 2, 'FuseP': 3, 'Mix': 1, 'NA': 1}

qual_cols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
for col in qual_cols:
    df_combined[col] = df_combined[col].map(qual_map)

df_combined['BsmtFinQuality1'] = df_combined['BsmtFinType1'].map(bsmt_fin_map)
df_combined['BsmtFinQuality2'] = df_combined['BsmtFinType2'].map(bsmt_fin_map)
df_combined['BsmtExposure'] = df_combined['BsmtExposure'].map(bsmt_exp_map)
df_combined['GarageFinish'] = df_combined['GarageFinish'].map(garage_fin_map)
df_combined['Functional'] = df_combined['Functional'].map(functional_map)
df_combined['PavedDrive'] = df_combined['PavedDrive'].map(paved_drive_map)
df_combined['Electrical'] = df_combined['Electrical'].map(electrical_map)

# 3.3 Создание признаков на основе условий
df_combined['Alley_road'] = df_combined['Alley'].map({'NA': 0, 'Grvl': 1, 'Pave': 2})
df_combined['LvlLotShape'] = df_combined['LotShape'].map({'Reg': 0, 'IR1': 1, 'IR2': 2, 'IR3': 3})
df_combined['LandContourLvl'] = df_combined['LandContour'].apply(lambda x: 0 if x == 'Lvl' else (1 if x == 'Low' else 2))
df_combined['Inside'] = (df_combined['LotConfig'] == 'Inside').astype(int)
df_combined['Corner'] = (df_combined['LotConfig'] == 'Corner').astype(int)
df_combined['CulDSac'] = (df_combined['LotConfig'] == 'CulDSac').astype(int)
df_combined['FR'] = df_combined['LotConfig'].apply(lambda x: 2 if x == 'FR3' else (1 if x == 'FR2' else 0))
df_combined['LandSlopeLvl'] = df_combined['LandSlope'].map({'Gtl': 0, 'Mod': 1, 'Sev': 2})
df_combined['RR'] = ((df_combined['Condition1'].str.contains('RR')) | (df_combined['Condition2'].str.contains('RR'))).astype(int)
df_combined['PosObj'] = ((df_combined['Condition1'].str.contains('Pos')) | (df_combined['Condition2'].str.contains('Pos'))).astype(int)
df_combined['StreetObj'] = ((df_combined['Condition1'].isin(['Artery', 'Feedr'])) | (df_combined['Condition2'].isin(['Artery', 'Feedr']))).astype(int)
df_combined['notFinishFloor'] = ((df_combined['HouseStyle'] == '1.5Unf') | (df_combined['HouseStyle'] == '2.5Unf')).astype(int)
df_combined['Split'] = df_combined['HouseStyle'].apply(lambda x: 2 if x == 'SLvl' else (1 if x == 'SFoyer' else 0))
df_combined['NumGarage'] = df_combined.apply(lambda row: 2 if row['GarageType'] == '2Types' or row['MiscFeature'] == 'Gar2' else 0 if row['GarageType'] == 'NA' else 1, axis=1)
df_combined['Privacy'] = df_combined['Fence'].apply(lambda x: 2 if x == 'GdPrv' else (1 if x in ['MnPrv', 'GdWo'] else 0))
df_combined['WoodFence'] = df_combined['Fence'].apply(lambda x: 2 if x == 'GdWo' else (1 if x == 'MnWw' else 0))
df_combined['Is_Rec_Room1'] = (df_combined['BsmtFinType1'] == 'Rec').astype(int)
df_combined['Is_Rec_Room2'] = (df_combined['BsmtFinType2'] == 'Rec').astype(int)

# 3.4 Группировка редких категорий и упрощение признаков
df_combined['Neighborhood'] = df_combined['Neighborhood'].apply(lambda x: 'Other' if x in ['Veenker', 'NPkVill', 'Blmngtn', 'Blueste'] else x)
df_combined['MSZoning'] = df_combined['MSZoning'].apply(lambda x: 'Other' if x in ['C (all)', 'NA', 'RH'] else x)
df_combined['BldgType'] = df_combined['BldgType'].apply(lambda x: 'Other' if x in ['Duplex', 'Twnhs', '2fmCon'] else x)
df_combined['RoofStyle'] = df_combined['RoofStyle'].apply(lambda x: 'Other' if x in ['Gambrel', 'Flat', 'Mansard', 'Shed'] else x)
df_combined['RoofMatl'] = df_combined['RoofMatl'].apply(lambda x: 'Other' if x in ["Tar&Grv", "ClyTile", "Membran", "Metal", "Roll", "WdShngl", "WdShake"] else x)
df_combined['MasVnrType'] = df_combined['MasVnrType'].apply(lambda x: 'Other' if x in ['NA', 'BrkCmn'] else x)
df_combined['Foundation'] = df_combined['Foundation'].apply(lambda x: 'Other' if x in ['Stone', 'Wood'] else x)
df_combined['Heating'] = df_combined['Heating'].apply(lambda x: 'Other' if x in ['GasW', 'Grav', 'Wall', 'OthW', 'Floor'] else x)
df_combined['GarageType'] = df_combined['GarageType'].apply(lambda x: 'Other' if x in ['Basment', '2Types', 'CarPort'] else x)
df_combined['MiscFeature'] = df_combined['MiscFeature'].apply(lambda x: 'ShedOrOther' if x in ['Gar2', 'Othr', 'TenC', 'Shed'] else x)
df_combined['SaleType'] = df_combined['SaleType'].apply(lambda x: 'Other' if x in ['ConLD', 'CWD', 'ConLI', 'ConLw', 'Oth', 'Con', 'NA'] else x)
df_combined['SaleCondition'] = df_combined['SaleCondition'].apply(lambda x: 'Other' if x in ['Alloca', 'AdjLand'] else x)

# 3.5 One-Hot Encoding для материалов экстерьера
main_materials = ["VinylSd", "MetalSd", "HdBoard", "Wd Sdng", "Plywood", "CemntBd", "BrkFace", "WdShing", "AsbShng", "Stucco"]
for material in main_materials:
    df_combined[material] = ((df_combined['Exterior1st'] == material) | (df_combined['Exterior2nd'] == material)).astype(int)
df_combined['ExtMat_Other'] = ((~df_combined['Exterior1st'].isin(main_materials)) | (~df_combined['Exterior2nd'].isin(main_materials))).astype(int)


# --- ЭТАП 4: ФИНАЛИЗАЦИЯ И РАЗДЕЛЕНИЕ ---

# 4.1 Удаляем исходные столбцы, которые были преобразованы или больше не нужны
cols_to_drop = [
    'Id', 'MSSubClass', 'TotalBsmtSF', 'Street', 'Utilities', 'Alley', 'LotShape',
    'LandContour', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType',
    'HouseStyle', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'Exterior1st', 'Exterior2nd',
    'BsmtFinType1', 'BsmtFinType2', 'GarageYrBlt', 'Fence', 'YrSold', 'MoSold'
]
df_processed = df_combined.drop(columns=cols_to_drop, errors='ignore')

# 4.2 Преобразуем оставшиеся категориальные столбцы в dummy-переменные
df_processed = pd.get_dummies(df_processed, drop_first=True)

# 4.3 Преобразуем столбцы типа 'bool' (True/False) в 'int' (1/0)
bool_columns = df_processed.select_dtypes(include=['bool']).columns
df_processed[bool_columns] = df_processed[bool_columns].astype(int)


# --- ЭТАП 5: РАЗДЕЛЕНИЕ НА TRAIN И TEST ---

# 5.1 Разделяем обработанный датафрейм обратно на train и test
df_train_proc = df_processed.iloc[:train_rows].copy()
df_test_proc = df_processed.iloc[train_rows:].copy()

# --- КОНЕЦ СКРИПТА ---
print("Предобработка завершена.")
print(f"Размер обработанного train датасета: {df_train_proc.shape}")
print(f"Размер обработанного test датасета:  {df_test_proc.shape}")
print(f"Целевая переменная 'SalePrice' сохранена отдельно, размер: {train_target.shape}")

Данные успешно загружены.
Предобработка завершена.
Размер обработанного train датасета: (1460, 124)
Размер обработанного test датасета:  (1459, 124)
Целевая переменная 'SalePrice' сохранена отдельно, размер: (1460,)


In [ ]:
# Ячейка 3.5: Определение "слабых" признаков для отбора

# --- 1. Объединяем обработанные признаки с целевой переменной ---
# Это нужно для расчета корреляции
df_with_target = df_train_proc.copy()
df_with_target['SalePrice'] = train_target

# --- 2. Рассчитываем корреляцию каждого признака с 'SalePrice' ---
correlations = df_with_target.corr()['SalePrice'].abs().sort_values()

# --- 3. Исключаем саму целевую переменную из списка ---
correlations = correlations.drop('SalePrice')

# --- 4. Сохраняем отсортированный список названий признаков ---
# Это наши кандидаты на удаление, от самого слабого к более сильному
weakest_features_sorted = correlations.index.tolist()

print("Определен рейтинг признаков по их корреляции с 'SalePrice'.")
print(f"Всего признаков-кандидатов на удаление: {len(weakest_features_sorted)}")
print("\nТоп-15 самых слабых признаков (наименьшая корреляция с ценой):")
for i, feature in enumerate(weakest_features_sorted[:15]):
    print(f"{i+1:2d}. {feature:<20} (Корреляция: {correlations[feature]:.4f})")

Определен рейтинг признаков по их корреляции с 'SalePrice'.
Всего признаков-кандидатов на удаление: 124

Топ-15 самых слабых признаков (наименьшая корреляция с ценой):
 1. FR                   (Корреляция: 0.0034)
 2. Corner               (Корреляция: 0.0041)
 3. BsmtFinQuality2      (Корреляция: 0.0061)
 4. Foundation_Other     (Корреляция: 0.0083)
 5. BsmtFinSF2           (Корреляция: 0.0114)
 6. Neighborhood_SawyerW (Корреляция: 0.0146)
 7. ExtMat_Other         (Корреляция: 0.0162)
 8. BsmtHalfBath         (Корреляция: 0.0168)
 9. LandContourLvl       (Корреляция: 0.0175)
10. ExterCond            (Корреляция: 0.0189)
11. RR                   (Корреляция: 0.0195)
12. MiscVal              (Корреляция: 0.0212)
13. MoSoldAgo            (Корреляция: 0.0213)
14. SaleType_Other       (Корреляция: 0.0223)
15. Neighborhood_NWAmes  (Корреляция: 0.0235)


### Ячейка 4: Подготовка данных для обучения


In [ ]:
X = df_train_proc
y = train_target

# Логарифмируем целевую переменную для стабилизации обучения
y_log = np.log1p(y)

# Разделение на обучающую и валидационную выборки
X_train, X_val, y_train_log, y_val_log = train_test_split(
    X, y_log, 
    test_size=0.2, 
    random_state=42 # для воспроизводимости результатов
)

print("Данные разделены на обучающую и валидационную выборки.")
print(f"X_train: {X_train.shape}, y_train: {y_train_log.shape}")
print(f"X_val:   {X_val.shape}, y_val:   {y_val_log.shape}")

Данные разделены на обучающую и валидационную выборки.
X_train: (1168, 124), y_train: (1168,)
X_val:   (292, 124), y_val:   (292,)


### Ячейка 5: Класс-обертка для данных (PyTorch Dataset)

In [ ]:
# Преобразуем данные pandas в формат тензоров PyTorch
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_log.values, dtype=torch.float32).unsqueeze(1)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_log.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(df_test_proc.values, dtype=torch.float32)

# Создание кастомного Dataset
class HousesDataset(Dataset):
    """
    Класс-обертка для данных о домах.
    Позволяет DataLoader'у эффективно работать с нашими тензорами.
    """
    def __init__(self, features, labels=None):
        self.features = features
        self.labels = labels
        # Если метки (labels) не переданы, значит, это тестовый набор
        self.is_test = labels is None
    
    def __len__(self):
        # Возвращает общее количество примеров в датасете
        return len(self.features)
    
    def __getitem__(self, idx):
        # Возвращает один пример (признаки и, если есть, метку) по индексу
        if self.is_test:
            return self.features[idx]
        else:
            return self.features[idx], self.labels[idx]

print("Класс HousesDataset и тензоры данных готовы к использованию.")

Класс HousesDataset и тензоры данных готовы к использованию.


### Ячейка 6: Динамическая архитектура модели

In [ ]:
# Ячейка 6: Динамическая архитектура модели (с выбором функции активации)

class DynamicRegressionModel(nn.Module):
    """
    Нейронная сеть, архитектура и функция активации которой 
    определяются входными параметрами.
    """
    def __init__(self, num_features, hidden_layers, dropout_rates, activation_fn_name):
        super().__init__()
        
        # НОВОЕ: Словарь для сопоставления имени функции с классом из PyTorch
        activation_functions = {
            'relu': nn.ReLU(),
            'gelu': nn.GELU(),
            'leaky_relu': nn.LeakyReLU(),
            'silu': nn.SiLU() # SiLU (или Swish) - еще один мощный вариант
        }
        
        # Получаем объект функции активации по имени
        activation_fn = activation_functions.get(activation_fn_name)
        if activation_fn is None:
            raise ValueError(f"Неизвестная функция активации: {activation_fn_name}")

        layers = []
        input_dim = num_features
        
        # Динамически создаем скрытые слои
        for i, (hidden_dim, dropout_rate) in enumerate(zip(hidden_layers, dropout_rates)):
            layers.append(nn.BatchNorm1d(input_dim))
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(activation_fn) # ИЗМЕНЕНИЕ: используем выбранную функцию
            layers.append(nn.Dropout(dropout_rate))
            input_dim = hidden_dim
            
        # Добавляем выходной слой
        layers.append(nn.BatchNorm1d(input_dim))
        layers.append(nn.Linear(input_dim, 1))
        
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

print("Архитектура модели теперь поддерживает выбор функции активации.")

Архитектура модели теперь поддерживает выбор функции активации.


### Ячейка 7: objective — Сердце эксперимента

In [ ]:
# Ячейка 7: objective — Сердце эксперимента (с отбором признаков)

def objective_for_optuna(trial):
    """
    "Чистая" функция для одного эксперимента Optuna. 
    Обучает модель с предложенными гиперпараметрами и возвращает метрику качества.
    """
    # --- 1. Предложение гиперпараметров от Optuna ---
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.004, 0.004, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [64]),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-1, log=True),
        'num_hidden_layers': trial.suggest_int('num_hidden_layers', 1, 1),
        'activation_fn': trial.suggest_categorical('activation_fn', ['gelu']),
        # НОВЫЙ ГИПЕРПАРАМЕТР: Сколько самых слабых признаков отбросить
        'num_cols_to_drop': trial.suggest_int('num_cols_to_drop', 0, 90, step=5)
    }
    
    hidden_layers = []
    dropout_rates = []
    for i in range(params['num_hidden_layers']):
        hidden_layers.append(trial.suggest_int(f'n_units_l{i}', 64, 64, step=16))
        dropout_rates.append(trial.suggest_float(f'dropout_l{i}', 0.0, 0.5, step=0.1))

    # --- 2. Отбор признаков для текущего trial ---
    # Получаем список колонок для удаления на основе предложенного Optuna числа
    num_to_drop = params['num_cols_to_drop']
    cols_to_drop = weakest_features_sorted[:num_to_drop] if num_to_drop > 0 else []
    
    # Создаем копии данных для этого trial и удаляем колонки
    X_train_trial = X_train.drop(columns=cols_to_drop)
    X_val_trial = X_val.drop(columns=cols_to_drop)
    
    # --- 3. Подготовка данных и модели для текущего trial ---
    # Преобразуем ИЗМЕНЕННЫЕ данные в тензоры
    X_train_tensor_trial = torch.tensor(X_train_trial.values, dtype=torch.float32)
    X_val_tensor_trial = torch.tensor(X_val_trial.values, dtype=torch.float32)

    train_loader = DataLoader(HousesDataset(X_train_tensor_trial, y_train_tensor), batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(HousesDataset(X_val_tensor_trial, y_val_tensor), batch_size=params['batch_size'], shuffle=False)

    model = DynamicRegressionModel(
        # ВАЖНО: количество признаков теперь динамическое
        num_features=X_train_trial.shape[1], 
        hidden_layers=hidden_layers, 
        dropout_rates=dropout_rates,
        activation_fn_name=params['activation_fn']
    ).to(DEVICE)
    
    optimizer = torch.optim.RAdam(model.parameters(), lr=params['learning_rate'], weight_decay=params['weight_decay'])
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=Config.SCHEDULER_FACTOR, patience=Config.SCHEDULER_PATIENCE)
    criterion = nn.MSELoss()

    # --- 4. Цикл обучения и валидации (без изменений) ---
    min_val_loss = float('inf')
    for epoch in range(EPOCHS):
        model.train()
        for features, labels in train_loader:
            features, labels = features.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        current_val_loss = 0.0
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(DEVICE), labels.to(DEVICE)
                outputs = model(features)
                loss = criterion(outputs, labels)
                current_val_loss += loss.item() * features.size(0)
        
        epoch_val_loss = np.sqrt(current_val_loss / len(val_loader.dataset))
        scheduler.step(epoch_val_loss)

        if epoch_val_loss < min_val_loss:
            min_val_loss = epoch_val_loss
            
        if (epoch + 1) % 60 == 0:
            trial.report(epoch_val_loss, epoch)
            
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
            
    return min_val_loss

print("Objective функция обновлена: теперь она оптимизирует и набор используемых признаков.")

Objective функция обновлена: теперь она оптимизирует и набор используемых признаков.


In [ ]:
# Ячейка 7.5: Создание нашего собственного, надежного TQDM Callback

from tqdm.notebook import tqdm

class MyTqdmCallback:
    """
    Кастомный callback для отображения прогресс-бара TQDM для экспериментов Optuna.
    """
    def __init__(self, n_trials):
        self.n_trials = n_trials
        self.pbar = tqdm(total=n_trials, desc="Optuna Optimization")

    def __call__(self, study: "optuna.study.Study", trial: "optuna.trial.FrozenTrial"):
        """
        Вызывается после каждого завершенного trial.
        """
        self.pbar.update(1)

    def close(self):
        """
        Закрывает прогресс-бар после завершения исследования.
        """
        self.pbar.close()

print("Создан кастомный MyTqdmCallback.")

Создан кастомный MyTqdmCallback.


In [ ]:
import optuna
import numpy as np
from optuna.pruners import BasePruner, MedianPruner
from optuna.trial import TrialState

class MovingAveragePruner(BasePruner):
    """
    Прунер-обертка, который принимает решение на основе скользящего среднего
    промежуточных значений, чтобы сгладить шум.
    """
    def __init__(self, base_pruner: BasePruner, window_size: int):
        self.base_pruner = base_pruner
        self.window_size = window_size

    def prune(self, study: "optuna.study.Study", trial: "optuna.trial.FrozenTrial") -> bool:
        intermediate_values = trial.intermediate_values
        steps = list(intermediate_values.keys())
        
        if len(steps) < self.window_size:
            return self.base_pruner.prune(study, trial)

        last_steps = sorted(steps)[-self.window_size:]
        moving_avg = np.mean([intermediate_values[step] for step in last_steps])
        
        # --- ИСПРАВЛЕНИЕ ---
        # Создаем "двойника", передавая ему все обязательные поля,
        # включая недостающий trial._trial_id.
        dummy_trial = optuna.trial.FrozenTrial(
            number=trial.number,
            state=trial.state,
            value=trial.value,
            datetime_start=trial.datetime_start,
            datetime_complete=trial.datetime_complete,
            params=trial.params,
            distributions=trial.distributions,
            user_attrs=trial.user_attrs,
            system_attrs=trial.system_attrs,
            intermediate_values={steps[-1]: moving_avg},
            trial_id=trial._trial_id  # <-- ВОТ ИСПРАВЛЕНИЕ
        )
        
        return self.base_pruner.prune(study, dummy_trial)

print("Создан кастомный прунер MovingAveragePruner (версия 2.0).")

Создан кастомный прунер MovingAveragePruner (версия 2.0).


### Ячейка 8: Запуск поиска и логирование только лучшего результата в MLflow

In [ ]:
# Ячейка 8: Запуск оптимизации и логирование лучшей модели в MLflow (ФИНАЛЬНАЯ ВЕРСИЯ)

# --- 1. Настройка и запуск исследования Optuna ---
storage_name = "sqlite:///optuna_study.db"
study_name = "house-prices-pytorch-feature-selection2" # Новое имя для чистоты эксперимента

mlflow.set_experiment(EXPERIMENT_NAME)

base_pruner = MedianPruner(n_startup_trials=8, n_warmup_steps=100)
smart_pruner = MovingAveragePruner(base_pruner=base_pruner, window_size=10)

study = optuna.create_study(
    study_name=study_name,
    storage=storage_name,
    direction='minimize',
    pruner=smart_pruner,
    load_if_exists=True
)

my_callback = MyTqdmCallback(N_TRIALS)
try:
    study.optimize(objective_for_optuna, n_trials=N_TRIALS, callbacks=[my_callback])
finally:
    my_callback.close()

# --- 2. Вывод результатов Optuna ---
print("\nОптимизация Optuna завершена!")
best_trial = study.best_trial
best_params = best_trial.params
print(f"Лучший trial: {best_trial.number}")
print(f"  Значение (min val_rmse): {best_trial.value:.4f}")
print("  Лучшие гиперпараметры: ")
for key, value in best_params.items():
    print(f"    {key}: {value}")

# --- 3. Логирование лучшего результата в MLflow ---
print("\nЗапись лучшего эксперимента в MLflow...")
with mlflow.start_run(run_name="Best_Run_With_Feature_Selection") as parent_run:
    
    mlflow.log_params(best_params)
    mlflow.log_metric("final_val_rmse", best_trial.value)
    mlflow.set_tag("optuna_trial_number", best_trial.number)
    
    print("Переобучение финальной модели на лучших параметрах и с лучшим набором признаков...")
    
    # --- 4. Воссоздание лучших условий ---
    # 4.1. Определяем, какие колонки нужно удалить
    num_to_drop = best_params['num_cols_to_drop']
    cols_to_drop = weakest_features_sorted[:num_to_drop] if num_to_drop > 0 else []
    
    print(f"Найдено, что лучший результат достигается при удалении {len(cols_to_drop)} признаков.")
    
    # 4.2. Готовим данные с оптимальным набором признаков
    X_train_final = X_train.drop(columns=cols_to_drop)
    X_train_final_tensor = torch.tensor(X_train_final.values, dtype=torch.float32)
    y_train_final_tensor = torch.tensor(y_train_log.values, dtype=torch.float32).unsqueeze(1)

    # 4.3. Собираем параметры для модели
    num_hidden_layers = best_params['num_hidden_layers']
    hidden_layers = [best_params[f'n_units_l{i}'] for i in range(num_hidden_layers)]
    dropout_rates = [best_params[f'dropout_l{i}'] for i in range(num_hidden_layers)]
    
    # 4.4. Создаем и обучаем финальную модель
    final_model = DynamicRegressionModel(
        num_features=X_train_final.shape[1], # ВАЖНО: правильное число признаков
        hidden_layers=hidden_layers,
        dropout_rates=dropout_rates,
        activation_fn_name=best_params['activation_fn']
    ).to(DEVICE)
    
    train_loader = DataLoader(HousesDataset(X_train_final_tensor, y_train_final_tensor), batch_size=best_params['batch_size'], shuffle=True)
    optimizer = torch.optim.RAdam(final_model.parameters(), lr=best_params['learning_rate'], weight_decay=best_params['weight_decay'])
    criterion = nn.MSELoss()
    
    # Простой цикл обучения без валидации, т.к. мы используем все данные
    for epoch in range(EPOCHS):
        for features, labels in train_loader:
            features, labels = features.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = final_model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    print("Финальная модель обучена.")
    
    # --- 5. Логирование артефакта-модели в MLflow ---
    # Важно, чтобы input_example имел правильную размерность
    input_example = X_train_final.head().values
    
    signature = mlflow.models.infer_signature(input_example)
    
    mlflow.pytorch.log_model(
        pytorch_model=final_model,
        artifact_path="model",
        signature=signature,
        input_example=input_example,
        registered_model_name="house-prices-regressor-best-fs" # fs = feature selection
    )
    # Логируем список удаленных колонок как артефакт для воспроизводимости
    pd.Series(cols_to_drop).to_csv("dropped_columns.txt", index=False, header=False)
    mlflow.log_artifact("dropped_columns.txt")
    
    print(f"Лучшая модель и список удаленных колонок сохранены в MLflow run_id: {parent_run.info.run_id}")

[I 2025-06-14 14:26:24,986] Using an existing study with name 'house-prices-pytorch-feature-selection2' instead of creating a new one.


Optuna Optimization:   0%|          | 0/200 [00:00<?, ?it/s]

[I 2025-06-14 14:26:38,958] Trial 200 finished with value: 0.11925028327486914 and parameters: {'learning_rate': 0.004, 'batch_size': 64, 'weight_decay': 0.0032228681714353133, 'num_hidden_layers': 1, 'activation_fn': 'gelu', 'num_cols_to_drop': 30, 'n_units_l0': 64, 'dropout_l0': 0.2}. Best is trial 113 with value: 0.11700463855647714.
[I 2025-06-14 14:26:46,606] Trial 201 pruned. 
[I 2025-06-14 14:26:52,474] Trial 202 pruned. 
[I 2025-06-14 14:26:58,457] Trial 203 pruned. 
[I 2025-06-14 14:27:04,458] Trial 204 pruned. 
[I 2025-06-14 14:27:16,715] Trial 205 finished with value: 0.11777805229937853 and parameters: {'learning_rate': 0.004, 'batch_size': 64, 'weight_decay': 0.0019747956655447397, 'num_hidden_layers': 1, 'activation_fn': 'gelu', 'num_cols_to_drop': 30, 'n_units_l0': 64, 'dropout_l0': 0.2}. Best is trial 113 with value: 0.11700463855647714.
[I 2025-06-14 14:27:21,808] Trial 206 pruned. 
[I 2025-06-14 14:27:27,616] Trial 207 pruned. 
[I 2025-06-14 14:27:33,541] Trial 208 pr


Оптимизация Optuna завершена!
Лучший trial: 305
  Значение (min val_rmse): 0.1165
  Лучшие гиперпараметры: 
    learning_rate: 0.004
    batch_size: 64
    weight_decay: 0.0018658713002051225
    num_hidden_layers: 1
    activation_fn: gelu
    num_cols_to_drop: 30
    n_units_l0: 64
    dropout_l0: 0.2

Запись лучшего эксперимента в MLflow...
Переобучение финальной модели на лучших параметрах и с лучшим набором признаков...
Найдено, что лучший результат достигается при удалении 30 признаков.


2025/06/14 14:51:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/14 14:51:37 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0a0+5228986c39.nv25.5) contains a local version label (+5228986c39.nv25.5). MLflow logged a pip requirement for this package as 'torch==2.8.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Финальная модель обучена.


2025/06/14 14:51:43 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0a0+5228986c39.nv25.5) contains a local version label (+5228986c39.nv25.5). MLflow logged a pip requirement for this package as 'torch==2.8.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/06/14 14:51:43 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": [
    [
      70.0,
      8400.0,
      5.0,
      6.0,
      0.0,
      3.0,
      3.0,
      3.0,
      1.0,
      922.0,
      392.0,
      3.0,
      0.0,
      1314.0,
      0.0,
      1314.0,
      1.0,
      1.0,
      0.0,
      3.0,
      1.0,
      3.0,
      5.0,
      0.0,
      0.0,
      0.0,
      2.0,
      1.0,
      294.0,
      3.0,
      3.0,
      0.0,
      250.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      53.0,
      53.0,
      53.0,
      3.5,
 

Лучшая модель и список удаленных колонок сохранены в MLflow run_id: 8c9a08638c554f37a69870a64fb8d930


Registered model 'house-prices-regressor-best-fs' already exists. Creating a new version of this model...
Created version '2' of model 'house-prices-regressor-best-fs'.



### Ячейка 9: Загрузка лучшей модели из MLflow и создание Submission

In [ ]:
# Ячейка 9: Загрузка лучшей модели из MLflow и создание Submission

print("Подготовка к созданию submission файла...")

# --- 1. Находим лучший run в MLflow ---
# Ищем по новому имени эксперимента, если вы его меняли
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
best_run = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=['metrics.final_val_rmse ASC']
).iloc[0]

print(f"Загрузка модели из MLflow run_id: {best_run.run_id}")

# --- 2. Загружаем модель и параметры ---
best_model_uri = f"runs:/{best_run.run_id}/model"
final_model = mlflow.pytorch.load_model(best_model_uri).to(DEVICE)
best_params = study.best_trial.params # Можно взять из study или из mlflow

# --- 3. Применяем отбор признаков к тестовым данным ---
# Определяем, какие колонки были удалены в лучшем trial
num_to_drop = best_params['num_cols_to_drop']
cols_to_drop = weakest_features_sorted[:num_to_drop] if num_to_drop > 0 else []

print(f"Применяем к тестовым данным удаление {len(cols_to_drop)} признаков...")

# Удаляем те же самые колонки из обработанного тестового датасета
df_test_final = df_test_proc.drop(columns=cols_to_drop)

# --- 4. Создание предсказаний ---
final_model.eval()
X_test_tensor = torch.tensor(df_test_final.values, dtype=torch.float32)
test_loader = DataLoader(HousesDataset(X_test_tensor), batch_size=256, shuffle=False)

all_predictions = []
with torch.no_grad():
    for features_batch in test_loader:
        features_batch = features_batch.to(DEVICE)
        outputs = final_model(features_batch)
        # Возвращаем к исходному масштабу цен
        predicted_prices = torch.expm1(outputs)
        all_predictions.extend(predicted_prices.cpu().numpy().flatten().tolist())

# --- 5. Создание файла для отправки ---
submission_df = pd.DataFrame({
    'Id': df_test_original['Id'],
    'SalePrice': all_predictions
})
submission_df.to_csv(SUBMISSION_FILE, index=False)

print(f"\nSubmission файл '{SUBMISSION_FILE}' успешно создан.")
print("Первые 5 строк submission файла:")
print(submission_df.head())

Подготовка к созданию submission файла...
Загрузка модели из MLflow run_id: 8c9a08638c554f37a69870a64fb8d930


Применяем к тестовым данным удаление 30 признаков...

Submission файл 'submission_house_prices.csv' успешно создан.
Первые 5 строк submission файла:
     Id      SalePrice
0  1461  112257.023438
1  1462  162489.828125
2  1463  176113.875000
3  1464  188928.250000
4  1465  198384.156250
